In [9]:
#Classification of Satellite Images by Tensorflow
#note that 

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import tensorflow_datasets as tfds

In [11]:
# import from local directory
data_dir = '/home/zheng/Desktop/seminar/image classification/cut_sub/'

In [12]:
# create dataset
batch_size = 50
img_height = 64
img_width = 64

# create training set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# create validation set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)# save to geotiff

# check class names
class_names = train_ds.class_names
print(class_names)

Found 1000 files belonging to 4 classes.
Using 800 files for training.
Found 1000 files belonging to 4 classes.
Using 200 files for validation.
['cloud', 'developed', 'green', 'water']


In [13]:
# rescale image to 0-1
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

# optimize for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
# train
num_classes = 4

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=16
)

Epoch 1/16
16/16 [==============================] - 3s 100ms/step - loss: 1.2235 - accuracy: 0.2862 - val_loss: 0.9816 - val_accuracy: 0.5850
Epoch 2/16
16/16 [==============================] - 1s 83ms/step - loss: 0.8526 - accuracy: 0.7125 - val_loss: 0.5801 - val_accuracy: 0.8350
Epoch 3/16
16/16 [==============================] - 1s 83ms/step - loss: 0.3486 - accuracy: 0.9125 - val_loss: 0.1877 - val_accuracy: 0.9550
Epoch 4/16
16/16 [==============================] - 1s 73ms/step - loss: 0.1469 - accuracy: 0.9438 - val_loss: 0.1678 - val_accuracy: 0.9700
Epoch 5/16
16/16 [==============================] - 1s 73ms/step - loss: 0.1817 - accuracy: 0.9362 - val_loss: 0.2569 - val_accuracy: 0.9350
Epoch 6/16
16/16 [==============================] - 1s 74ms/step - loss: 0.1694 - accuracy: 0.9413 - val_loss: 0.1331 - val_accuracy: 0.9800
Epoch 7/16
16/16 [==============================] - 1s 72ms/step - loss: 0.1096 - accuracy: 0.9688 - val_loss: 0.1563 - val_accuracy: 0.9500
Epoch 8/16
1

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0